In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import time
from simpletransformers.classification import ClassificationModel
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from simpletransformers.classification import ClassificationModel, ClassificationArgs 
from sklearn.metrics import accuracy_score 

c:\anaconda\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url_list = ["https://www.bloomberght.com/tum-ekonomi-haberleri",
    "https://www.bloomberght.com/tum-tarim-haberleri",
    "https://www.bloomberght.com/tum-finansal-teknoloji-haberleri",
    "https://www.bloomberght.com/tum-ekonomik-veriler-ve-gundem-haberleri",
    "https://www.bloomberght.com/tum-enerji-haberleri"]

In [3]:
news_date = []
news_title = []
news_content = []
news_cat = []
news_url_list = []

In [4]:
for page_number in range(15):
        for list in tqdm(url_list):
            cat = list.split('/tum-')[1].split('-haberleri')[0]
            time.sleep(10)
            url = urlopen(list + "/" + str(page_number)).read()
            soup = BeautifulSoup(url, 'html.parser')
            container = soup.find_all('div', class_='widget-news-list type1')
            news_order = len(container[0].find_all('li'))

            for i in range(news_order):
                try:
                    news_url = container[0].find_all('li')[i].find('a')['href']
                    new_news = 'https://www.bloomberght.com' + news_url
                    url1 = urlopen(new_news).read()
                    soup1 = BeautifulSoup(url1, 'html.parser')
                    container1 = soup1.find_all('div', class_='news-item')
                    news_date.append(container1[0].find('span', class_='date').text.split('\n')[1])
                    news_title.append(container1[0].find('h1', class_='title').text.split('\n')[1])
                    p_tags = container1[0].find('article', class_="content")('p')
                    text_list = [p.get_text(strip=True) for p in p_tags]
                    news_content.append('\n'.join(text_list))
                    news_cat.append(cat)
                except:
                     continue


100%|██████████| 5/5 [00:55<00:00, 11.16s/it]


In [5]:
news = {
     'title' : news_title,
     'date' : news_date,
     'content' : news_content,
     'category' : news_cat
}
df = pd.DataFrame(news)

In [6]:
df

,title,date,content,category
0,"Midas, 45 milyon dolar yatırım aldı","20 Nisan 2024 Cumartesi, 19:57",HANDE BERKTAN\nAmerikanBorsalarına veBorsa İst...,ekonomi
1,"TOÇEV 30. yılında 8,3 milyon öğrenciye ulaşıyor","20 Nisan 2024 Cumartesi, 13:17",TOÇEV Kurucusu ve Yönetim Kurulu Başkanı Ebru ...,ekonomi
2,ABD'nin petrol sondaj kulesi sayısı yükseldi,"20 Nisan 2024 Cumartesi, 11:01",Petrol sahası hizmetleri şirketi Baker Hughes'...,ekonomi
3,7 başlıkla geçen hafta,"20 Nisan 2024 Cumartesi, 10:49","Altın, İsrail ile İran arasında artan gerilime...",ekonomi
4,"BOJ'dan faiz artırımında ""kuvvetle muhtemel"" v...","20 Nisan 2024 Cumartesi, 03:08",\nJaponya Merkez Bankası (BOJ) Başkanı Kazuo U...,ekonomi
...,...,...,...,...
460,Yellen: İran'a yaptırım yetkimizi kullanmaktan...,"16 Nisan 2024 Salı, 19:45","Yellen, Uluslararası Para Fonu (IMF) ve Dünya ...",ekonomi
461,IMF'den merkez bankalarına 'finansal kırılganl...,"16 Nisan 2024 Salı, 19:23","Uluslararası Para Fonu (IMF), Küresel Finansal...",ekonomi
462,Cumhurbaşkanı Erdoğan: 200 yıllık tartışma bitti,"16 Nisan 2024 Salı, 18:54",Cumhurbaşkanı Recep Tayyip Erdoğan başkanlığın...,ekonomi
463,"Hazine 5,6 milyar liralık kira sertifikası ihr...","16 Nisan 2024 Salı, 18:23",Hazine ve Maliye Bakanlığı'nın internet sitesi...,ekonomi


In [7]:
def clean_text(text):
        """Cleans text by removing mentions, hashtags, URLs, and other unwanted patterns.

        Args:
        text (str): The text to be cleaned.

        Returns:
        str: The cleaned text.
        """
        text = re.sub(r'@[A-Za-z0-9]+', '', str(text))
        text = re.sub(r'#', '', str(text))
        text = re.sub(r'RT[\s]+', '', str(text))
        text = re.sub(r'https?:\/\/\S+', '', str(text))
        text = re.sub(r'\n', ' ', str(text))
        text = re.sub(r'[\xa0]+', '', str(text))
        return text

In [8]:
df['content'] = df['content'].apply(clean_text)

In [9]:
encoder = LabelEncoder()
df['labels'] = encoder.fit_transform(df['category'])

In [10]:
df

,title,date,content,category,labels
0,"Midas, 45 milyon dolar yatırım aldı","20 Nisan 2024 Cumartesi, 19:57",HANDE BERKTAN AmerikanBorsalarına veBorsa İsta...,ekonomi,0
1,"TOÇEV 30. yılında 8,3 milyon öğrenciye ulaşıyor","20 Nisan 2024 Cumartesi, 13:17",TOÇEV Kurucusu ve Yönetim Kurulu Başkanı Ebru ...,ekonomi,0
2,ABD'nin petrol sondaj kulesi sayısı yükseldi,"20 Nisan 2024 Cumartesi, 11:01",Petrol sahası hizmetleri şirketi Baker Hughes'...,ekonomi,0
3,7 başlıkla geçen hafta,"20 Nisan 2024 Cumartesi, 10:49","Altın, İsrail ile İran arasında artan gerilime...",ekonomi,0
4,"BOJ'dan faiz artırımında ""kuvvetle muhtemel"" v...","20 Nisan 2024 Cumartesi, 03:08",Japonya Merkez Bankası (BOJ) Başkanı Kazuo Ue...,ekonomi,0
...,...,...,...,...,...
460,Yellen: İran'a yaptırım yetkimizi kullanmaktan...,"16 Nisan 2024 Salı, 19:45","Yellen, Uluslararası Para Fonu (IMF) ve Dünya ...",ekonomi,0
461,IMF'den merkez bankalarına 'finansal kırılganl...,"16 Nisan 2024 Salı, 19:23","Uluslararası Para Fonu (IMF), Küresel Finansal...",ekonomi,0
462,Cumhurbaşkanı Erdoğan: 200 yıllık tartışma bitti,"16 Nisan 2024 Salı, 18:54",Cumhurbaşkanı Recep Tayyip Erdoğan başkanlığın...,ekonomi,0
463,"Hazine 5,6 milyar liralık kira sertifikası ihr...","16 Nisan 2024 Salı, 18:23",Hazine ve Maliye Bakanlığı'nın internet sitesi...,ekonomi,0


In [11]:
regex_tokenizer = nltk.RegexpTokenizer(r"\w+")
turkish_stop_words = stopwords.words('turkish')
dataset_x = []
for index, row in df.iterrows():
    tokenized_words = regex_tokenizer.tokenize(row['content'])
    tokenized_words = [word for word in tokenized_words if word not in turkish_stop_words]
    dataset_x.append(" ".join(tokenized_words))
df['content_processed'] = dataset_x

In [12]:
df

,title,date,content,category,labels,content_processed
0,"Midas, 45 milyon dolar yatırım aldı","20 Nisan 2024 Cumartesi, 19:57",HANDE BERKTAN AmerikanBorsalarına veBorsa İsta...,ekonomi,0,HANDE BERKTAN AmerikanBorsalarına veBorsa İsta...
1,"TOÇEV 30. yılında 8,3 milyon öğrenciye ulaşıyor","20 Nisan 2024 Cumartesi, 13:17",TOÇEV Kurucusu ve Yönetim Kurulu Başkanı Ebru ...,ekonomi,0,TOÇEV Kurucusu Yönetim Kurulu Başkanı Ebru Uyg...
2,ABD'nin petrol sondaj kulesi sayısı yükseldi,"20 Nisan 2024 Cumartesi, 11:01",Petrol sahası hizmetleri şirketi Baker Hughes'...,ekonomi,0,Petrol sahası hizmetleri şirketi Baker Hughes ...
3,7 başlıkla geçen hafta,"20 Nisan 2024 Cumartesi, 10:49","Altın, İsrail ile İran arasında artan gerilime...",ekonomi,0,Altın İsrail İran arasında artan gerilime iliş...
4,"BOJ'dan faiz artırımında ""kuvvetle muhtemel"" v...","20 Nisan 2024 Cumartesi, 03:08",Japonya Merkez Bankası (BOJ) Başkanı Kazuo Ue...,ekonomi,0,Japonya Merkez Bankası BOJ Başkanı Kazuo Ueda ...
...,...,...,...,...,...,...
460,Yellen: İran'a yaptırım yetkimizi kullanmaktan...,"16 Nisan 2024 Salı, 19:45","Yellen, Uluslararası Para Fonu (IMF) ve Dünya ...",ekonomi,0,Yellen Uluslararası Para Fonu IMF Dünya Bankas...
461,IMF'den merkez bankalarına 'finansal kırılganl...,"16 Nisan 2024 Salı, 19:23","Uluslararası Para Fonu (IMF), Küresel Finansal...",ekonomi,0,Uluslararası Para Fonu IMF Küresel Finansal İs...
462,Cumhurbaşkanı Erdoğan: 200 yıllık tartışma bitti,"16 Nisan 2024 Salı, 18:54",Cumhurbaşkanı Recep Tayyip Erdoğan başkanlığın...,ekonomi,0,Cumhurbaşkanı Recep Tayyip Erdoğan başkanlığın...
463,"Hazine 5,6 milyar liralık kira sertifikası ihr...","16 Nisan 2024 Salı, 18:23",Hazine ve Maliye Bakanlığı'nın internet sitesi...,ekonomi,0,Hazine Maliye Bakanlığı nın internet sitesinde...


In [13]:
train, test = train_test_split(df[['content_processed','labels']], test_size=0.3, random_state=0)

In [14]:
train.columns = ['text','labels']
test.columns = ['text','labels']

In [15]:
model_args = ClassificationArgs(num_train_epochs=10,overwrite_output_dir=True, train_batch_size = 64)
model = ClassificationModel('bert', "dbmdz/bert-base-turkish-128k-uncased", num_labels=5, use_cuda=False,
                                    args=model_args)
model.train_model(train)
preds = []
for i in test['text']:
    predictions, raw_outputs = model.predict([i])
    preds.append(predictions[0])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
1it [00:03,  3.20s/it]
Epoch 10 of 10: 100%|██████████| 10/10 [13:04<00:00, 78.46s/it]
1it [00:05,  5.56s/it]
100%|██████████| 1/1 [00:00<00:00,  9.60it/s]
1it [00:02,  2.84s/it]
100%|██████████| 1/1 [00:00<00:00, 11.29it/s]
1it [00:02,  2.80s/it]
100%|██████████| 1/1 [00:00<00:00, 13.01it/s]
1it [00:02,  2.83s/it]
100%|██████████| 1/1 [00:00<00:00, 12.84it/s]
1it [00:02,  2.84s/it]
100%|██████████| 1/1 [00:00<00:00, 12.84it/s]
1it [00:02,  2.85s/it]
100%|██████████| 1/1 [00:00<00:00, 13.00it/s]
1it [00:02,  2.93s/it]
100%|██████████| 1/1 [00:00<00:00, 13.00it/s]
1it [00:02,  2.95s/it]
100%|██████████| 1/1 [00:00<00:00, 12.68it/s]
1it [00:02,  2.88s/it]
100%|██████████| 1/1 [00:0

In [17]:
accuracy_score(test['labels'], preds)

0.8